## Change Working Directory

The notebooks are stored in a subfolder, therefore when running the notebook in the editor, we need to change the working directory from its current folder to its parent folder

We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'c:\\Users\\Bartek\\Desktop\\Predictive-Analysis\\jupyter_notebooks'

- We use os.path.dirname() to get the parent directory
- Then we call the os.chir() function, which defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

# Load Data

In [3]:
import pandas as pd
loan_data_drop = pd.read_csv("outputs/loan_data_drop.csv")
loan_data_mean = pd.read_csv("outputs/loan_data_mean.csv")

databases = [loan_data_drop, loan_data_mean]

We noticed that the database consists of objects and some float/int values. We will need to convert these values so that we can run a machine learning algorithms on the data.

In [4]:
for loan_data in databases:
    loan_data['Married'] = loan_data['Married'].map({'No': 0, 'Yes': 1})
    loan_data['Gender'] = loan_data['Gender'].map({'Female': 0, 'Male': 1})
    loan_data['Education'] = loan_data['Education'].map({'Not Graduate': 0, 'Graduate': 1})
    loan_data['Self_Employed'] = loan_data['Self_Employed'].map({'No': 0, 'Yes': 1})
    loan_data['Property_Area'] = loan_data['Property_Area'].map({'Rural': 0, 'Semiurban': 1, 'Urban': 2})
    loan_data['Loan_Status'] = loan_data['Loan_Status'].map({'N': 0, 'Y': 1})
    loan_data['Dependents'] = loan_data['Dependents'].map({'0': 0, '1' : 1, '2': 1, '3+': 2})
    loan_data.info()
    loan_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381 entries, 0 to 380
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            381 non-null    object 
 1   Gender             381 non-null    int64  
 2   Married            381 non-null    int64  
 3   Dependents         381 non-null    int64  
 4   Education          381 non-null    int64  
 5   Self_Employed      381 non-null    int64  
 6   ApplicantIncome    381 non-null    int64  
 7   CoapplicantIncome  381 non-null    float64
 8   LoanAmount         381 non-null    float64
 9   Loan_Amount_Term   381 non-null    float64
 10  Credit_History     381 non-null    float64
 11  Property_Area      381 non-null    int64  
 12  Loan_Status        381 non-null    int64  
dtypes: float64(4), int64(8), object(1)
memory usage: 38.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381 entries, 0 to 380
Data columns (total 13 columns):
 #   Co

In [5]:
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)

resampled_databases = []
for data in databases:
    data.drop(columns=['Loan_ID'], inplace=True)
    X = data.drop(columns=['Loan_Status'])
    y = data['Loan_Status']
    X_resampled, y_resampled = ros.fit_resample(X, y)
    resampled_data = pd.concat([pd.DataFrame(X_resampled),
                                pd.DataFrame(y_resampled,columns=['Loan_Status'])], axis=1)
    resampled_databases.append(resampled_data)


Resample datasets and evaluate the performance of multiple classifiers, including Logistic Regression, Decision Tree, Random Forest, Support Vector Machine, and K-Nearest Neighbors, on each dataset. I will prepare the data by extracting features and target variables, splits the dataset into training and testing sets, trains each classifier on the training data, and evaluates its accuracy on the test set. If a classifier achieves an accuracy greater than 0.8, it prints out the classifier's name and its corresponding accuracy. This process helps identify classifiers that perform well on imbalanced datasets, particularly after applying resampling techniques to address class imbalances.

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

classifiers = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Support Vector Machine": SVC(),
    "K-Nearest Neighbors": KNeighborsClassifier()
}



for data in resampled_databases:
    for _ in range(10):
        X = data.drop(columns=['Loan_Status'])  # Assuming 'target_column' is your target variable
        y = data['Loan_Status']  # Assuming 'target_column' is your target variable

        # Split the dataset into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Define a list of classifiers to test


        # Train and test each classifier
        for clf_name, clf in classifiers.items():
            #print(f"Training and testing {clf_name}...")
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            if accuracy > 0.8:
                print(f"Accuracy of {clf_name}: {accuracy:.4f}")
                print("-" * 50)
                print(classification_report(y_test, y_pred))

c:\Users\Bartek\Desktop\Predictive-Analysis\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy of Decision Tree: 0.8991
--------------------------------------------------
              precision    recall  f1-score   support

           0       0.88      0.93      0.90        55
           1       0.92      0.87      0.90        54

    accuracy                           0.90       109
   macro avg       0.90      0.90      0.90       109
weighted avg       0.90      0.90      0.90       109

Accuracy of Random Forest: 0.9083
--------------------------------------------------
              precision    recall  f1-score   support

           0       0.89      0.93      0.91        55
           1       0.92      0.89      0.91        54

    accuracy                           0.91       109
   macro avg       0.91      0.91      0.91       109
weighted avg       0.91      0.91      0.91       109

Accuracy of Decision Tree: 0.8899
--------------------------------------------------
              precision    recall  f1-score   support

           0       0.86      0.93   

c:\Users\Bartek\Desktop\Predictive-Analysis\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy of Random Forest: 0.9174
--------------------------------------------------
              precision    recall  f1-score   support

           0       0.91      0.93      0.92        55
           1       0.92      0.91      0.92        54

    accuracy                           0.92       109
   macro avg       0.92      0.92      0.92       109
weighted avg       0.92      0.92      0.92       109

Accuracy of Decision Tree: 0.9083
--------------------------------------------------
              precision    recall  f1-score   support

           0       0.89      0.93      0.91        55
           1       0.92      0.89      0.91        54

    accuracy                           0.91       109
   macro avg       0.91      0.91      0.91       109
weighted avg       0.91      0.91      0.91       109



c:\Users\Bartek\Desktop\Predictive-Analysis\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

c:\Users\Bartek\Desktop\Predictive-Analysis\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy of Random Forest: 0.9174
--------------------------------------------------
              precision    recall  f1-score   support

           0       0.91      0.93      0.92        55
           1       0.92      0.91      0.92        54

    accuracy                           0.92       109
   macro avg       0.92      0.92      0.92       109
weighted avg       0.92      0.92      0.92       109

Accuracy of Decision Tree: 0.9083
--------------------------------------------------
              precision    recall  f1-score   support

           0       0.89      0.93      0.91        55
           1       0.92      0.89      0.91        54

    accuracy                           0.91       109
   macro avg       0.91      0.91      0.91       109
weighted avg       0.91      0.91      0.91       109



c:\Users\Bartek\Desktop\Predictive-Analysis\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy of Random Forest: 0.9266
--------------------------------------------------
              precision    recall  f1-score   support

           0       0.93      0.93      0.93        55
           1       0.93      0.93      0.93        54

    accuracy                           0.93       109
   macro avg       0.93      0.93      0.93       109
weighted avg       0.93      0.93      0.93       109

Accuracy of Decision Tree: 0.9083
--------------------------------------------------
              precision    recall  f1-score   support

           0       0.89      0.93      0.91        55
           1       0.92      0.89      0.91        54

    accuracy                           0.91       109
   macro avg       0.91      0.91      0.91       109
weighted avg       0.91      0.91      0.91       109

Accuracy of Random Forest: 0.9358
--------------------------------------------------
              precision    recall  f1-score   support

           0       0.94      0.93   

c:\Users\Bartek\Desktop\Predictive-Analysis\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
